In [ ]:
#load packages
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import timeit
import random
import sys
sys.path.append('../../../../DataScience/lib/')
import pymei
from dev.bind_prod_utils import utils as dev_utils
import numpy as np

In [ ]:
# starburst access credentials. create connection
username = input('Enter your jumpcloud username: ')
print('Enter your password:')
conn = pymei.connect.presto(username=username, phi=True, starburst=True)

In [ ]:
#load data: 
surname_query = """
        select
            coverage_core.member_id as "member_id"
            , upper(bind_vw_latest_member.last_name) as "surname"
        from hive.data_layer_prod.bind_vw_member_coverage as coverage_core
        left join member.public.member  AS member ON coverage_core.member_id = member.id
        left join hive.data_layer_prod.bind_vw_latest_member as bind_vw_latest_member ON member.id = bind_vw_latest_member.id
        where 1=1 
            and coverage_core.is_core_coverage = true 
            and coverage_core.is_active_core_member = true 
            and coverage_core.is_current_plan_year = true
        group by 1,2
    """
member_surname = pd.read_sql(surname_query, conn)

In [ ]:
S3_PATH = 's3://bind-prod-data-science-analytics/alexl/'
census_surname = pd.read_csv(S3_PATH + 'predict_race/Names_2010Census.csv')

In [ ]:
member_surname=member_surname.rename(columns={'member_id': 'member_id', 'surname': 'name'})

In [ ]:
member_surname

In [ ]:
census_surname.head()

In [ ]:
census_surname = census_surname.iloc[:, 0:2]
census_surname

In [ ]:
left_join_df=pd.merge(member_surname, census_surname, on='name', how='left')
left_join_df

In [ ]:
na_df = left_join_df[left_join_df['pctwhite'].isna()]
na_df

In [ ]:
census_surname['name'] = census_surname['name'].astype(str)
census_surname['name'

In [ ]:
mylist = census_surname['name'].astype(str).values.tolist()
pattern = '|'.join(mylist)

In [ ]:
import re
def pattern_searcher(search_str:str, search_list:str):

    search_obj = re.search(search_list, search_str)
    if search_obj :
        return_str = search_str[search_obj.start(): search_obj.end()]
    else:
        return_str = 'NA'
    return return_str

In [ ]:
na_df['matched_str'] = na_df['name'].apply(lambda x: pattern_searcher(search_str=x, search_list=pattern))

In [ ]:
na_df

In [ ]:
na_df.to_csv ('/home/jupyter-leo.zhou/DataScience/projects/RA_equity_OKR/restricted /na_df.csv')